In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [3]:
load_dotenv(override=True)

True

agent will ask the MCP client what tools are available, it selects the right one, it uses it by passing the page url

all communications happents over stdio using MCP protocol

In [5]:
# Fetch mcp-server -> enables LLMs to retrieve and process content from web pages

fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]} #runs this command on my computer

# create mcp client running within openai agents sdk
async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as server:
    fetch_tools = await server.list_tools()

for tool in fetch_tools:
    print(f"{tool.name}: {tool.description.replace('\n', ' ')}")

fetch: Fetches a URL from the internet and optionally extracts its contents as markdown.  Although originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.


In [6]:
playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as server:
    playwright_tools = await server.list_tools()

playwright_tools

[Tool(name='browser_close', description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True)),
 Tool(name='browser_resize', description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True)),
 Tool(name='browser_console_messages', description='Returns all console messages', inputSchema={'type': 'object', 'properties': {}, 'additi

In [7]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', description='Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=None),
 Tool(name='read_multiple_files', description="Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.", inputSchema={'type': 'object', 'properties': {'paths': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['pat

### Agent with Tools!

In [8]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as mcp_server_browser:
        agent = Agent(
            name="investigator",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
        )
        
        with trace('investigate'):
            result = await Runner.run(agent, "Find a great recipe for Carrot Cake, then summarize it in markdown to cake.md")
            print(result.final_output)
            

It seems I'm unable to save files directly due to directory restrictions. However, here’s a summarized carrot cake recipe in markdown format:

```markdown
# Carrot Cake Recipe

## Ingredients
- **Vegetable cooking spray**
- **2 cups whole wheat pastry flour**
- **1 3/4 teaspoons baking soda**
- **1/2 teaspoon salt**
- **1 1/2 teaspoons pumpkin pie spice**
- **3 large eggs**
- **1 cup granulated sugar**
- **1/2 cup light brown sugar**
- **1/3 cup vegetable oil**
- **3/4 cup buttermilk**
- **1 teaspoon pure vanilla extract**
- **3 cups coarsely grated carrots (about 1 pound)**
  
### For the Cream Cheese Frosting:
- **12 ounces reduced-fat cream cheese**
- **1 cup icing sugar**
- **1 1/2 teaspoons vanilla extract**

## Instructions
1. Preheat the oven to 175°C (350°F). Spray a 9x13-inch baking pan with vegetable spray; line it with parchment paper, leaving an overhang on two sides and spray the paper.
2. Whisk together the flour, baking soda, salt, and pumpkin pie spice in a medium bowl.